![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# "Plan your trip with Kayak"
## _Data Collection and Management Project_

### Le Projet

L'équipe marketing de Kayak a découvert que 70 % de leurs utilisateurs qui planifient un voyage aimeraient avoir plus d'informations sur la destination vers laquelle ils se rendent.
Par conséquent, l'équipe souhaite créer une application qui recommandera aux gens où planifier leurs prochaines vacances en se basant sur deux variables :
* Météo
* Hôtels dans la région

Le projet débutant à peine, notre travail consiste à :
* Récupérer les coordonnées GPS des destinations
* Obtenir des données météorologiques pour chaque destination
* Obtenir des informations sur les hôtels pour chaque destination
* Stocker toutes les informations ci-dessus dans un _data lake_
* Extraire, transformer et charger les données nettoyées du _data lake_ vers un _data warehouse_

### 1. Récupérer les données météo

Grâce à `nominatim.streetmap.org`, nous allons pouvoir obtenir les coordonnées GPS d'une sélection de villes.

In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv("./.env"))

Ci-dessous la liste des villes que l'équipe de Kayak veut analyser pour commencer.

In [ ]:
CITIES =[
    "Mont Saint Michel",
    "St Malo",
    "Bayeux",
    "Le Havre",
    "Rouen",
    "Paris",
    "Amiens",
    "Lille",
    "Strasbourg",
    "Chateau du Haut Koenigsbourg",
    "Colmar",
    "Eguisheim",
    "Besancon",
    "Dijon",
    "Annecy",
    "Grenoble",
    "Lyon",
    "Gorges du Verdon",
    "Bormes les Mimosas",
    "Cassis",
    "Marseille",
    "Aix en Provence",
    "Avignon",
    "Uzes",
    "Nimes",
    "Aigues Mortes",
    "Saintes Maries de la mer",
    "Collioure",
    "Carcassonne",
    "Ariege",
    "Toulouse",
    "Montauban",
    "Biarritz",
    "Bayonne",
    "La Rochelle"
]

In [5]:
def get_cities_coordinates(cities: list[str]) -> list[dict]:
    url = "https://nominatim.openstreetmap.org"
    endpoint = "search"
    data = []
    for city in cities:
        city = city.lower()
        payload = {"city": city, "format": "json"}
        response = requests.get(f"{url}/{endpoint}", params=payload)

        if response.status_code != 200:
            print(f"Failed with '{city}' -> {response.status_code}")
            continue

        resp = response.json()[0]
        data.append({"city": city,
                        "lat": resp.get("lat", None),
                        "lon": resp.get("lon", None)})

    return data

Toujours via `requests`, nous allons cette fois-ci récupérer des données sur la météo à venir dans une sélection de villes.

Pour cela, utilisons l'API `openweathermap.org`. La stratégie reste la même si ce n'est que cette API a besoin d'une API KEY pour executer des requêtes.

Nous avons décidé de récupérer les données météorologiques des 7 jours à venir.
Le but est de déterminer dans quelles villes le temps sera le plus agréable.
Pour cela, nous allons nous baser sur ces éléments :
* la température minimum (`temp_min`)
* la température maximum (`temp_max`)
* le pourcentage d'humidité (`humidity`)
* le pourcentage de couverture nuageuse (`clouds`)
* la probabilité qu'il pleuve (`rain_prob`)

In [6]:
def get_weather_data(cities_coordinates: list[dict]) -> list[dict]:
    url = "https://api.openweathermap.org/data/2.5"
    endpoint = "forecast"
    data = []
    city_id = 1
    for row in cities_coordinates.copy():
        payload = {"lat": float(row["lat"]),
                    "lon": float(row["lon"]),
                    "units": "metric",
                    "cnt": 7,  # 7 days to come
                    "appid": os.getenv("WEATHER_KEY")}

        response = requests.get(f"{url}/{endpoint}",
                                params=payload)

        if response.status_code != 200:
            print(f"Failed -> {response.status_code}")
            continue

        day_id = 1
        for dt in response.json()["list"]:
            data.append({
                "city_id": city_id,
                "city": row["city"],
                "lat": float(row["lat"]),
                "lon": float(row["lon"]),
                "day_id": day_id,
                "temp": dt["main"].get("temp", None),
                "temp_min": dt["main"].get("temp_min", None),
                "temp_max": dt["main"].get("temp_max", None),
                "humidity": dt["main"].get("humidity", None),
                "clouds": dt["clouds"].get("all", None),
                "rain_prob": dt.get("pop", None),
            })
            day_id += 1
        city_id += 1

    return data

Maintenant que les fonctions sont écrites, il n'y a plus qu'à enregistrer le résultat dans un fichier CSV !
_(Pour des raisons de gains de place, les conventions PEP8 ne sont pas respectées dans la cellule ci-dessous)._

In [ ]:
df = pd.DataFrame(get_weather_data(get_cities_coordinates(CITIES)))
df.to_csv("weather_data.csv", index=False, encoding="utf-8")

### 2. Récupérer les données sur les hôtels

Le client veut se baser sur le site booking.com pour collecter ses données. Comme il n'y a pas d'API officielle, il va falloir faire du web scraping pour obtenir les informations dont nous avons besoin.

Nous avons donc simulé une recherche sur le site et récupéré tous les hôtels proposés grâce au CSS.
Après avoir stocké leur nom, note, et adresse URL, nous avons simulé un clic sur les hôtels pour accéder à leurs détails (voir méthode `self.parse_hotel_detail`).


In [ ]:
import scrapy


class BookingSpider(scrapy.Spider):
    name = "booking"
    allowed_domains = ["booking.com"]
    cities = ["montpellier"]
    start_urls = [f"https://www.booking.com/searchresults.html?ss={city}" for city in CITIES]

    custom_settings = {
        "USER_AGENT": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "ROBOTSTXT_OBEY": False
    }

    def parse(self, response):
        hotels = response.css('div[data-testid="property-card"]')

        for hotel in hotels:
            city = response.url.split("=")[-1]
            name = hotel.css('div[data-testid="title"]::text').get().strip()
            rating = hotel.css("div.f13857cc8c.e008572b71::text").get()
            if not rating:
                rating = "0.0"
            url = hotel.css('a[data-testid="title-link"]::attr(href)').get()

            yield response.follow(url, callback=self.parse_hotel_detail,
                                  meta={"name": name,
                                        "rating": rating,
                                        "city": city
                                        })

        next_page = response.css("a.bui-pagination__link.pagenext::attr(href)").get()
        if next_page:
            yield response.follow(next_page, self.parse)

    def parse_hotel_detail(self, response):
        city = response.meta["city"]
        name = response.meta["name"]
        url = response.url
        rating = response.meta["rating"]
        address = response.css("span.hp_address_subtitle.js-hp_address_subtitle.jq_tooltip::text").get().strip()
        description = response.css('p[data-testid="property-description"]::text').get().strip()
        coordinates = response.css("a#hotel_address::attr(data-atlas-latlng)").get()

        if coordinates:
            latitude, longitude = coordinates.split(",")
            latitude = float(latitude)
            longitude = float(longitude)
        else:
            latitude = None
            longitude = None

        yield {
            "city": city,
            "name": name,
            "url": url,
            "rating": rating,
            "address": address,
            "description": description,
            "latitude": latitude,
            "longitude": longitude
        }

Nous avons stocké les données dans un fichier JSON, il faut donc les convertir en CSV.

In [ ]:
with open("hotels.json", "r", encoding="utf-8") as file:
    hotel_content = file.read()

df = pd.DataFrame(eval(hotel_content))
df.to_csv("csv_files/weather_data.csv", index=False, encoding="utf-8")